In [1]:
import network as layer
import torch.nn as nn

nc = 3
nz = 512

In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
lr     = 0.001
beta1  = 0.0     
beta2  = 0.99     
imageSize = 128
batchSize = 16

outf = "./celeba_result/"
des_dir = "./celeba/"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([
                               transforms.CenterCrop(224),
                               transforms.Resize(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [3]:
from custom_layers import *

class Generator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.deconv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Generator, self).__init__()
        options = {'leaky':True, 'bn':False, 'wn':True, 'pixel':True}
        layers = []
        layers.append(layer.deconv(nz, 48, 4, 1, 3, **options))
        
        layers.append(Dense(layer.deconv(48, 24, 3, 1, 1, **options))) #(24 -> 36)
        layers.append(Dense(layer.deconv(72, 24, 3, 1, 1, **options))) #(36 -> 48)
        layers.append(Dense(layer.deconv(96, 24, 3, 1, 1, **options))) #(48 -> 60)
        #4x4
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(Dense(layer.deconv(120, 24, 3, 1, 1, **options))) #(60 -> 72)
        layers.append(Dense(layer.deconv(144, 24, 3, 1, 1, **options))) #(72 -> 84)
        layers.append(Dense(layer.deconv(168, 24, 3, 1, 1, **options))) #(84 -> 96)
        
        layers.append(layer.deconv(192, 96, 3, 1, 1, **options)) #(96 -> 48)
        layers.append(layer.deconv(96, 48, 3, 1, 1, **options)) #(48 -> 24)   
        #8x8
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(Dense(layer.deconv(48, 24, 3, 1, 1, **options))) #(24 -> 36)
        layers.append(Dense(layer.deconv(72, 24, 3, 1, 1, **options))) #(36 -> 48)
        layers.append(Dense(layer.deconv(96, 24, 3, 1, 1, **options))) #(48 -> 60)
        #4x4
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(Dense(layer.deconv(120, 24, 3, 1, 1, **options))) #(60 -> 72)
        layers.append(Dense(layer.deconv(144, 24, 3, 1, 1, **options))) #(72 -> 84)
        layers.append(Dense(layer.deconv(168, 24, 3, 1, 1, **options))) #(84 -> 96)
        
        layers.append(layer.deconv(192, 96, 3, 1, 1, **options)) #(96 -> 48)
        layers.append(layer.deconv(96, 48, 3, 1, 1, **options)) #(48 -> 24)  
        #32x32
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(Dense(layer.deconv(48, 24, 3, 1, 1, **options))) #(24 -> 36)
        layers.append(Dense(layer.deconv(72, 24, 3, 1, 1, **options))) #(36 -> 48)
        layers.append(Dense(layer.deconv(96, 24, 3, 1, 1, **options))) #(48 -> 60)
        #4x4
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(Dense(layer.deconv(120, 24, 3, 1, 1, **options))) #(60 -> 72)
        layers.append(Dense(layer.deconv(144, 24, 3, 1, 1, **options))) #(72 -> 84)
        layers.append(Dense(layer.deconv(168, 24, 3, 1, 1, **options))) #(84 -> 96)
        
        layers.append(layer.deconv(192, 96, 3, 1, 1, **options)) #(96 -> 48)
        layers.append(layer.deconv(96, 48, 3, 1, 1, **options)) #(48 -> 24)  
        
        layers.append(layer.deconv(48, nc, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=True, only=True))
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
netG = Generator()

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        options = {'leaky':True, 'bn':False, 'wn':True, 'pixel':False, 'gdrop':True}
        
        layers = []
        layers.append(layer.conv(nc, 48, 1, 1, 0, **options)) #(nc -> 24)
        
        layers.append(Dense(layer.conv(48, 24, 3, 1, 1, **options))) #(24 -> 36)
        layers.append(Dense(layer.conv(72, 24, 3, 1, 1, **options))) #(36 -> 48)
        layers.append(Dense(layer.conv(96, 24, 3, 1, 1, **options))) #(48 -> 60)
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        #64 x 64
        layers.append(Dense(layer.conv(120, 24, 3, 1, 1, **options))) #(60 -> 72)
        layers.append(Dense(layer.conv(144, 24, 3, 1, 1, **options))) #(72 -> 84)
        layers.append(Dense(layer.conv(168, 24, 3, 1, 1, **options))) #(84 -> 96)
        layers.append(nn.AvgPool2d(kernel_size=2))
        #32 x 32
        layers.append(Dense(layer.conv(192, 24, 3, 1, 1, **options))) #(96 -> 108)
        layers.append(Dense(layer.conv(216, 24, 3, 1, 1, **options))) #(108 -> 102)
        layers.append(Dense(layer.conv(240, 24, 3, 1, 1, **options))) #(120 -> 132)
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        #16 x 16
        layers.append(Dense(layer.conv(264, 24, 3, 1, 1, **options))) #(132 -> 144)
        layers.append(Dense(layer.conv(288, 24, 3, 1, 1, **options))) #(144 -> 156)
        layers.append(Dense(layer.conv(312, 24, 3, 1, 1, **options))) #(156 -> 168)
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        #8 x 8
        layers.append(Dense(layer.conv(336, 24, 3, 1, 1, **options))) #(168 -> 180)
        layers.append(Dense(layer.conv(360, 24, 3, 1, 1, **options))) #(180 -> 192)
        layers.append(Dense(layer.conv(384, 24, 3, 1, 1, **options))) #(192 -> 204)
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        #4 x 4
        layers.append(Dense(layer.conv(408, 24, 3, 1, 1, **options))) #(204 -> 216)
        layers.append(Dense(layer.conv(432, 24, 3, 1, 1, **options))) #(216 -> 228)
        layers.append(Dense(layer.conv(456, 24, 3, 1, 1, **options))) #(228 -> 240)
        
        layers.append(minibatch_std_concat_layer())
        layers.append(layer.conv(480 + 1, 480, 3, 1, 1, **options))
        layers.append(layer.conv(480, 480, 4, 1, 0, **options))
        layers.append(layer.linear(480, 1, sig=False, wn=True))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)
    
netD = Discriminator()

In [5]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils

In [6]:
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label_real = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
input, label_real, label_fake = input.cuda(), label_real.cuda(), label_fake.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
label_real.resize_(batchSize, 1).fill_(1)
label_fake.resize_(batchSize, 1).fill_(0)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
print()

In [7]:
fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
schedulerD = optim.lr_scheduler.MultiStepLR(optimizerD, milestones=[4, 7, 11, 17], gamma=0.87)
schedulerG = optim.lr_scheduler.MultiStepLR(optimizerG, milestones=[4, 7, 11, 17], gamma=0.87)
result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [ ]:
import numpy as np
_d_ = None
def add_noise(x, d_fake):
    global _d_
    if _d_ is not None:
        _d_ = _d_ * 0.9 + torch.mean(d_fake).data[0] * 0.1
        strength = 0.2 * max(0, _d_ - 0.5)**2
        z = np.random.randn(*x.size()).astype(np.float32) * strength
        z = Variable(torch.from_numpy(z)).cuda()
        return x + z
    else:
        _d_ = 0.0
        return x


In [ ]:
niter = 100
d_fake_save = None
for epoch in range(niter):
    schedulerD.step()
    schedulerG.step()
    for i, data in enumerate(dataloader, 0):

        # train D
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)

        inputv = Variable(input)
        inputv = add_noise(inputv, d_fake_save)
        
        d_real = netD(inputv)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        
        d_fake_save = d_fake = netD(fake.detach())
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%250 == 0:
            fake = netG(fixed_noise)
            image = fake.data.mul_(2).add_(0.5)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))

[0/100][0/12663] Loss_D: 1.0065 Loss_G: 0.8456 D(x): -0.0032 D(G(z)): -0.0053
[0/100][250/12663] Loss_D: 0.5099 Loss_G: 0.2344 D(x): 0.3273 D(G(z)): 0.2334
[0/100][500/12663] Loss_D: 0.3765 Loss_G: 0.4738 D(x): 0.6177 D(G(z)): 0.4683
[0/100][750/12663] Loss_D: 0.5991 Loss_G: 0.3290 D(x): 0.2638 D(G(z)): 0.2261
[0/100][1000/12663] Loss_D: 0.4057 Loss_G: 0.7633 D(x): 0.7909 D(G(z)): 0.5825
[0/100][1250/12663] Loss_D: 0.3975 Loss_G: 0.4438 D(x): 0.6175 D(G(z)): 0.4909
[0/100][1500/12663] Loss_D: 0.3116 Loss_G: 0.4804 D(x): 0.7072 D(G(z)): 0.4572
[0/100][1750/12663] Loss_D: 0.5843 Loss_G: 0.4661 D(x): 0.2519 D(G(z)): 0.0666
[0/100][2000/12663] Loss_D: 0.4086 Loss_G: 0.3627 D(x): 0.5229 D(G(z)): 0.3803
[0/100][2250/12663] Loss_D: 0.2994 Loss_G: 0.6142 D(x): 0.6705 D(G(z)): 0.3963
[0/100][2500/12663] Loss_D: 0.3638 Loss_G: 0.5205 D(x): 0.5720 D(G(z)): 0.3586
[0/100][2750/12663] Loss_D: 0.4430 Loss_G: 0.4740 D(x): 0.5202 D(G(z)): 0.4470
[0/100][3000/12663] Loss_D: 0.6679 Loss_G: 1.1393 D(x): 